In [7]:
import requests
import pandas as pd
import time
import networkx as nx

In [8]:
#list of ORFs obtained from SGD, read in as dataframe

orfdf = pd.read_csv(open("SGD_ORF.txt", 'rb'), sep='\t')

#orf names placed in list for ease
orfs = list(orfdf["Feature Name"])

#for each ORF, regulators are obtained from server and placed into dataframe

SGD_BASE_URL = 'https://www.yeastgenome.org/webservice'


In [12]:
#for each ORF, obtain interactors from server and place into dataframe. Label each interactor by their interaction type

time0 = time.time()
interactors1 = []
interactors2 = []
interactortype = []
i=0
for orf in orfs:
    
    url = 'https://www.yeastgenome.org/backend/locus/' + orf + '/interaction_details'
    go = True
    while go == True:
        try:
            response = requests.get(url=url).json()
            go = False
        except ValueError:
            print("decode error")
            time0 = time.time()
            while time.time()-time0<1:
                a=1
            pass
    orfinteractors1 = []
    orfinteractors2 = []
    orfinteractortype = []
    for value in response:
        orfinteractors1.append(value['locus1']['format_name'])
        orfinteractors2.append(value['locus2']['format_name'])
        if "Genetic" in value['experiment']['display_name']:
            orfinteractortype.append(value['experiment']['display_name'])
        else:
            orfinteractortype.append(value['interaction_type'])
    interactors1.append(orfinteractors1)
    interactors2.append(orfinteractors2)
    interactortype.append(orfinteractortype)
    time0 = time.time()
    i+=1
    while time.time()-time0<1:
        a=1
    
    
orfdf["interactors_type"] = interactortype
orfdf['interactors1'] = interactors1
orfdf['interactors2'] = interactors2

featurename = list(orfdf['Feature Name'])
allinteractors = []
for i in range(0,len(interactors1)):
    orfinteractors = []
    for j in range(0,len(interactors1[i])):
        test = 0
        if interactors1[i][j] != featurename[i]:
            orfinteractors.append(interactors1[i][j])
            test+=1
        if interactors2[i][j] != featurename[i]:
            orfinteractors.append(interactors2[i][j])
            test+=1
        if test > 1:
            print("Error at i:" + str(i)+ " j:" + str(j))
    allinteractors.append(orfinteractors)
orfdf['interactors'] = allinteractors
orfdf.to_pickle("orfdf_both_interactions.pickle")

decode error
decode error
decode error
decode error
decode error
decode error
decode error
decode error


In [9]:
orfdf = pd.read_pickle("orfdf_both_interactions.pickle")

In [10]:
categories = []
with open('enzrxns.dat') as file:
    for line in file:
        if line.strip()[0] != '#' and line.strip()[0] != r'/' and line.strip()[0] != '^':
            categories.append(line.strip().split(" - ")[0])

categories = list(set(categories))

entry = -1
table = []
for value in categories:
    table.append([])
with open('enzrxns.dat') as file:
    for line in file:
        #print(line)
        if line.strip()[0] == r'/'or entry == -1:
            if len(line.strip())>1:
                if line.strip()[1] == r'/':
                    entry += 1
                    for value in table:
                        value.append([])
        elif line.strip()[0] != '#' and line.strip()[0] != r'/' and line.strip()[0] != '^':
            table[categories.index(line.strip().split(" - ")[0])][entry].append(line.strip().split(" - ")[1])
            
enzrxnsdf = pd.DataFrame(table)
enzrxnsdf = enzrxnsdf.transpose()

enzrxnsdf.columns = categories

enzrxnsdf.to_csv("enzreactions.csv")
enzrxnsdf.to_pickle("enzreactions.pickle")

In [11]:
categories = []
with open('reactions.dat') as file:
    for line in file:
        if line.strip()[0] != '#' and line.strip()[0] != r'/' and line.strip()[0] != '^':
            categories.append(line.strip().split(" - ")[0])
categories = list(set(categories))
entry = -1
table = []
for value in categories:
    table.append([])
with open('reactions.dat') as file:
    for line in file:
        #print(line)
        if line.strip()[0] == r'/'or entry == -1:
            if len(line.strip())>1:
                if line.strip()[1] == r'/':
                    entry += 1
                    for value in table:
                        value.append([])
        elif line.strip()[0] != '#' and line.strip()[0] != r'/' and line.strip()[0] != '^':
            table[categories.index(line.strip().split(" - ")[0])][entry].append(line.strip().split(" - ")[1])
            
reactionsdf = pd.DataFrame(table)
reactionsdf = reactionsdf.transpose()

reactionsdf.columns = categories

reactionsdf.to_csv("reactions.csv")
reactionsdf.to_pickle("reactions.pickle")

In [12]:
categories = []
with open('compounds.dat') as file:
    for line in file:
        if line.strip()[0] != '#' and line.strip()[0] != r'/' and line.strip()[0] != '^':
            categories.append(line.strip().split(" - ")[0])
categories = list(set(categories))
entry = -1
table = []
for value in categories:
    table.append([])
with open('compounds.dat') as file:
    for line in file:
        #print(line)
        if line.strip()[0] == r'/'or entry == -1:
            if len(line.strip())>1:
                if line.strip()[1] == r'/':
                    entry += 1
                    for value in table:
                        value.append([])
        elif line.strip()[0] != '#' and line.strip()[0] != r'/' and line.strip()[0] != '^':
            if len(line.strip().split(" - ")) > 1:
                table[categories.index(line.strip().split(" - ")[0])][entry].append(line.strip().split(" - ")[1])
            
compoundsdf = pd.DataFrame(table)
compoundsdf = compoundsdf.transpose()

compoundsdf.columns = categories

compoundsdf.to_csv("compounds.csv")
compoundsdf.to_pickle("compounds.pickle")

In [13]:
enzymesdf = pd.read_csv(open("enzymes.col.txt", 'rb'), sep='\t')

subunits = list(enzymesdf['SUBUNIT-COMPOSITION'])

i = 0
while i < len(subunits):
    if (type(subunits[i]) == type("")):
        subunitslist = subunits[i].split(",")
        j = 0
        while j < len(subunitslist):
            if "-MONOMER" in subunitslist[j]: 
                subunitslist[j]=subunitslist[j].split("-MONOMER")[0]
            subunitslist[j]=subunitslist[j].split('*')[1]
            j+=1
        subunits[i] = subunitslist
    else:
        subunits[i] = []
    i+=1
    
    
enzymesdf['subunits'] = subunits
enzymerxnid = enzymesdf["UNIQUE-ID"]

genesdf = pd.read_csv(open("genes.col.txt", 'rb'), sep='\t')
geneid = list(genesdf["UNIQUE-ID"])

allgene_enzymerxns = []
i = 0
while i < len(geneid):
    j = 0
    gene_enzymerxns = []
    while j < len(subunits):
        k=0
        while k < len(subunits[j]):
            if geneid[i] == subunits[j][k]:
                gene_enzymerxns.append(enzymerxnid[j])
            k+=1
        j+=1
    allgene_enzymerxns.append(gene_enzymerxns)
    i+=1
    
genesdf["Enzyme Reactions"] = allgene_enzymerxns


right = list(reactionsdf["RIGHT"])
left = list(reactionsdf["LEFT"])
reactionsID = list(reactionsdf["UNIQUE-ID"])
enzymereactions = list(reactionsdf["ENZYMATIC-REACTION"])

i = 0
finalright = []
finalleft = []
finalreactionsID = []
while i < len(allgene_enzymerxns):
    j = 0
    tempright = []
    templeft = []
    tempreact = []
    while j < len(allgene_enzymerxns[i]):
        k = 0
        while k < len(enzymereactions):
            l = 0
            while l < len(enzymereactions[k]):
                if allgene_enzymerxns[i][j] == enzymereactions[k][l]:
                    tempright.append(right[k])
                    templeft.append(left[k])
                    tempreact.append(reactionsID[k])
                    
                l+=1
            k+=1
        j+=1
    finalright.append(tempright)
    finalleft.append(templeft)
    finalreactionsID.append(tempreact)
    i+=1
    
genesdf["ReactionID"] = finalreactionsID
genesdf["Left"] = finalleft
genesdf["Right"] = finalright

genesdf.to_csv("genesdf.csv")
genesdf.to_pickle("genesdf.pickle")

In [14]:
orfs = list(orfdf["Feature Name"])
names = list(orfdf["Standard Gene Name"])
alias = list(orfdf["Alias"])
#regulators = list(orfdf["Regulators"])
#yeastract = list(orfdf["yeastract"])
interactors = list(orfdf["interactors"])
interactors_type = list(orfdf["interactors_type"])
#interactome = list(orfdf["interactome"])

names[609]="OCT1"

genesdfname = genesdf['NAME']
left = genesdf['Left']
right = genesdf['Right']

newleft = []
newright = []

for value in orfs:
    newleft.append(0)
    newright.append(0)
    
i = 0
while i < len(names):
    if type(names[i]) != type(""):
        names[i] = orfs[i]
        if (type(names[i]) != type(1.0)):
            print(names[i])
    i+=1

i = 0
while i < len(alias):
    if type(alias[i]) != type(""):
        alias[i] = ""
    i+=1
    
i = 0
while i < len(genesdfname):
    found = False
    j = 0
    while j < len(orfs):
        if genesdfname[i] == orfs[j] or genesdfname[i] == names[j] or (genesdfname[i] in alias[j].split("|")):
            newleft[j] = left[i]
            newright[j] = right[i]
            found = True
            j=len(orfs)
        j+=1
    if found == False:
        orfs.append(genesdfname[i])
        names.append(genesdfname[i])
        alias.append(genesdfname[i])
        #regulators.append([])
        #yeastract.append([])
        interactors.append([])
        interactors_type.append([])
        #interactome.append([])
        newleft.append(left[i])
        newright.append(right[i])
        print("cant find" + str(genesdfname[i]))
    i+=1

YAL069W
YAL068W-A
YAL067W-A
YAL066W
YAL065C
YAL064W-B
YAL064W
YAL063C-A
YAL059C-A
YAL056C-A
YAL047W-A
YAL045C
YAL042C-A
YAL037C-B
YAL037C-A
YAL037W
YAL034C-B
YAL031W-A
YAL026C-A
YAL019W-A
YAL016C-B
YAL016C-A
YAL004W
YAR019W-A
YAR023C
YAR028W
YAR029W
YAR030C
YAR035C-A
YAR047C
YAR053W
YAR060C
YAR064W
YAR066W
YAR068W
YAR069C
YAR070C
YAR075W
YJL225W-A
YJL225C
YJL222W-B
YJL222W-A
YJL220W
YJL218W
YJL215C
YJL213W
YJL211C
YJL206C
YJL202C
YJL197C-A
YJL195C
YJL193W
YJL182C
YJL175W
YJL169W
YJL163C
YJL156W-A
YJL152W
YJL150W
YJL144W
YJL136W-A
YJL135W
YJL133C-A
YJL132W
YJL127W-A
YJL127C-B
YJL120W
YJL118W
YJL119C
YJL107C
YJL086C
YJL077W-B
YJL077W-A
YJL070C
YJL068C
YJL067W
YJL064W
YJL055W
YJL052C-A
YJL047C-A
YJL045W
YJL043W
YJL032W
YJL028W
YJL027C
YJL026C-A
YJL022W
YJL020W-A
YJL015C
YJL009W
YJL007C
YJR011C
YJR012C
YJR015W
YJR018W
YJR020W
YJR023C
YJR037W
YJR039W
YJR038C
YJR056C
YJR061W
YJR071W
YJR079W
YJR084W
YJR085C
YJR087W
YJR096W
YJR098C
YJR112W-A
YJR114W
YJR115W
YJR120W
YJR124C
YJR128W
YJR140W-A
YJ

YIR023C-A
YIR030W-A
YIR035C
YIR036W-A
YIR040C
YIR042C
cant findEgh1
cant findFLO8
cant findCRS5
cant findYMR050C
cant findSNR85
cant findSNR5
cant findYPL257W-A
cant findYDR098C-A
cant findYFL002W-B
cant findYGR161W-A
cant findRUF5-2
cant findSNR128
cant findYLR035C-A
cant findSNR11
cant findSNR66
cant findSNR31
cant findYPR137C-B
cant findYAR009C
cant findYCL019W
cant findYDR210W-B
cant findYER160C
cant findRUF21
cant findYGR027W-B
cant findSNR68
cant findSNR64
cant findYLR157C-B
cant findSNR74
cant findYNL054W-A
cant findYOR192C-A
cant findYPR158W-A
cant findHRA1
cant findSNR43
cant findYDR261W-A
cant findSNR67
cant findSNR48
cant findSNR10
cant findPWR1
cant findSNR38
cant findYLR256W-A
cant findSNR77
cant findNME1
cant findSNR81
cant findYBL100W-A
cant findYDR316W-B
cant findSNR4
cant findSNR46
cant findYHR214C-B
cant findYJR026W
cant findSNR87
cant findSNR30
cant findSNR24
cant findSNR50
cant findSNR59
cant findRME2
cant findYBR012W-B
cant findSNR47
cant findYDR210C-C
cant findYER

cant findtA(AGC)H
cant findtI(AAU)I2
cant findtR(ACG)J
cant findtK(UUU)K
cant findtI(UAU)L
cant findtE(UUC)M
cant findSUP8
cant findtD(GUC)O
cant findtC(GCA)B
cant findtC(GCA)P2
cant findSUP53
cant findtD(GUC)Q
cant findtK(CUU)D1
cant findtS(UGA)Q2
cant findtQ(UUG)D2
cant findtG(GCC)E
cant findtF(GAA)F
cant findtE(UUC)G3
cant findtL(GAG)G
cant findtH(GUG)H
cant findtT(AGU)I1
cant findtR(UCU)J2
cant findtN(GUU)K
cant findtL(UAA)L
cant findtH(GUG)M
cant findSUF6
cant findSUF17
cant findtN(GUU)O1
cant findtF(GAA)B
cant findtF(GAA)P2
cant findAAD16
cant findSUF2
cant findtG(UCC)Q
cant findtV(UAC)Q
cant findtR(UCU)D
cant findtI(AAU)E1
cant findtK(CUU)F
cant findtG(GCC)G2
cant findtR(UCU)G2
cant findtS(AGA)H
cant findtD(GUC)J1
cant findtV(AAC)J
cant findtV(AAC)K1
cant findtN(GUU)L
cant findSUF7
cant findtL(CAA)N
cant findSUF11
cant findtL(UAA)B1
cant findtI(AAU)P1
cant findtT(AGU)C
cant findtK(UUU)Q
cant findtS(AGA)D3
cant findtK(CUU)E2
cant findtS(GCU)F
cant findtH(GUG)G2
cant findtT(UGU)G1

In [26]:
connectionsdf = pd.DataFrame(list(zip(names, orfs, alias, interactors, interactors_type, newleft, newright)), 
                              columns =["Name", "ORF", "Alias","SGDInteractors", "InteractorsType", "ReactionLeft", 'ReactionRight'])

In [27]:
connectionsdf.to_pickle("connectionsdf.pickle")
connectionsdf.to_csv("connectionsdf.csv")


In [15]:
connectionsdf = pd.read_pickle("connectionsdf.pickle")

In [16]:
connectionsdf[100:125]

,Name,ORF,Alias,SGDInteractors,InteractorsType,ReactionLeft,ReactionRight
100,YAR030C,YAR030C,,[],[],0,0
101,MST28,YAR033W,DUP240 family protein MST28,"[YGL051W, YIL109C, YGL051W, YGL051W, YPR113W, ...","[Physical, Physical, Physical, Physical, Physi...",[],[]
102,YAT1,YAR035W,carnitine O-acetyltransferase YAT1,"[YAL054C, YER024W, YBL032W, YGR218W, YDL160C, ...","[Physical, Physical, Physical, Physical, Physi...","[[CARNITINE, ACETYL-COA]]","[[O-ACETYLCARNITINE, CO-A]]"
103,YAR035C-A,YAR035C-A,,"[YDL013W, YBR011C, YDR086C, YDR182W, YDR339C, ...","[Positive Genetic, Negative Genetic, Negative ...",[],[]
104,SWH1,YAR042W,oxysterol-binding protein related protein SWH1...,"[YLR397C, YGR192C, YER120W, YHR195W, YLR191W, ...","[Physical, Physical, Physical, Physical, Physi...",[],[]
105,YAR047C,YAR047C,,[],[],0,0
106,FLO1,YAR050W,flocculin FLO1|FLO4|FLO2,"[YBL032W, YGL122C, YMR080C, YBR245C, YER118C, ...","[Physical, Physical, Physical, Physical, Physi...",[],[]
107,YAR053W,YAR053W,,[],[],0,0
108,YAR060C,YAR060C,,[],[],0,0
109,YAR064W,YAR064W,,[YBR109C],[Physical],[],[]


In [17]:
orfs = list(connectionsdf["ORF"])
names = list(connectionsdf["Name"])
alias = list(connectionsdf["Alias"])
#regulators = list(connectionsdf["SGDRegulators"])
#yeastract = list(connectionsdf["YeastractRegulators"])
interactors = list(connectionsdf["SGDInteractors"])
interactors_type = list(connectionsdf["InteractorsType"])
#interactome = list(connectionsdf["InteractomeInteractors"])
left = list(connectionsdf["ReactionLeft"])
right = list(connectionsdf["ReactionRight"])

i = 0
while i < len(left):
    if left[i] == 0:
        left[i] = []
    if right[i] == 0:
        right[i] = []
        
    i+=1

In [39]:
G = nx.Graph()
G1 = nx.Graph()
i = 0
G.add_node('CIT', form = 'compound', distance = int(0))
while i < len(orfs):
    try:
        G.add_node(orfs[i], form = 'orf', name = names[i], distance = int(10*round(nx.dijkstra_path_length(G1, orfs[i], 'CIT'), 1)))
    except:
        G.add_node(orfs[i], form = 'orf', name = names[i], distance = -1)
        
    k = 0
    while k < len(left[i]):
        l = 0
        while l < len(left[i][k]):
            try:
                G.add_node(left[i][k][l], form = 'compound', distance = int(10*round(nx.dijkstra_path_length(G1, left[i][k][l], 'CIT'),1)))
            except:
                G.add_node(left[i][k][l], form = 'compound', distance = -1)
                
            G.add_edge(orfs[i], left[i][k][l], label = "backward_rxn", weight = int(10*0.3))
            #print("add edge")
            l+=1
        k+=1
    
          
    k = 0
    while k < len(right[i]):
        l = 0
        while l < len(right[i][k]):
            try:
                G.add_node(right[i][k][l], form = 'compound', distance = int(10*round(nx.dijkstra_path_length(G1, right[i][k][l], 'CIT'),1)))
            except:
                G.add_node(right[i][k][l], form = 'compound', distance = -1)
            G.add_edge(orfs[i], right[i][k][l], label = "forward_rxn", weight = int(10*0.3))
            #print("ADD EDGE")
            l+=1
        k+=1
    
    j=0
    #print(i/len(orfs))
    #nx.write_graphml(G,"metabolism_test.xml")
    print (i /len(orfs))
    i+=1
G.remove_node("PROTON")
G.remove_node("WATER")
G.remove_node("ATP")
G.remove_node("ADP")
G.remove_node("PPI")
G.remove_node("Pi")
G.remove_node("Protein-L-serine-or-L-threonine")
G.remove_node("Protein-Ser-or-Thr-phosphate")
G.remove_node("AMP")
G.remove_node("NAD")
G.remove_node("NADH")
G.remove_node("CO-A")
G.remove_node("NADP")
G.remove_node("NADPH")
G.remove_node("CARBON-DIOXIDE")
G.remove_node("GLT")
G.remove_node("S-ADENOSYLMETHIONINE")
G.remove_node("OXYGEN-MOLECULE")
G.remove_node("ACETYL-COA")
G.remove_node("AMMONIUM")
G.remove_node("ADENOSYL-HOMO-CYS")
G.remove_node("Nucleoside-Triphosphates")
G.remove_node("Peptides-holder")
G.remove_node("RNA-Holder")
G.remove_node("Cytochromes-C-Oxidized")
G.remove_node("Cytochromes-C-Reduced")
G.remove_node("GDP")
G.remove_node("Ubiquitin-C-Terminal-Glycine")
G.remove_node("General-Protein-Substrates")

i = 0

while i < len(orfs):
    k = 0
    while k < len(interactors[i]):
        if interactors_type[i][k] == "Positive Genetic":
            G.add_edge(names[i], interactors[i][k], label = interactors_type[i][k], weight = int(10))
 
        k+=1

    i+=1
    print (i /len(orfs))

import copy
G1 = copy.deepcopy(G)


G = nx.Graph()
i = 0
G.add_node('CIT', form = 'compound', distance = int(0))
while i < len(orfs):
    try:
        G.add_node(orfs[i], form = 'orf', name = names[i], distance = int(10*round(nx.dijkstra_path_length(G1, orfs[i], 'CIT'), 1)))
    except:
        G.add_node(orfs[i], form = 'orf', name = names[i], distance = -1)
    k = 0
    while k < len(left[i]):
        l = 0
        while l < len(left[i][k]):
            try:
                G.add_node(left[i][k][l], form = 'compound', distance = int(10*round(nx.dijkstra_path_length(G1, left[i][k][l], 'CIT'),1)))
            except:
                G.add_node(left[i][k][l], form = 'compound', distance = -1)
            G.add_edge(orfs[i], left[i][k][l], label = "backward_rxn", weight = 10*0.3)
            #print("add edge")
            l+=1
        k+=1
    
          
    k = 0
    while k < len(right[i]):
        l = 0
        while l < len(right[i][k]):
            try:
                G.add_node(right[i][k][l], form = 'compound', distance = int(10*round(nx.dijkstra_path_length(G1, right[i][k][l], 'CIT'),1)))
            except:
                G.add_node(right[i][k][l], form = 'compound', distance = -1)
            G.add_edge(orfs[i], right[i][k][l], label = "forward_rxn", weight = int(10*0.3))
            #print("ADD EDGE")
            l+=1
        k+=1
    
    j=0
    #print(i/len(orfs))
    #nx.write_graphml(G,"metabolism_test.xml")
    print (i /len(orfs))
    i+=1
G.remove_node("PROTON")
G.remove_node("WATER")
G.remove_node("ATP")
G.remove_node("ADP")
G.remove_node("PPI")
G.remove_node("Pi")
G.remove_node("Protein-L-serine-or-L-threonine")
G.remove_node("Protein-Ser-or-Thr-phosphate")
G.remove_node("AMP")
G.remove_node("NAD")
G.remove_node("NADH")
G.remove_node("CO-A")
G.remove_node("NADP")
G.remove_node("NADPH")
G.remove_node("CARBON-DIOXIDE")
G.remove_node("GLT")
G.remove_node("S-ADENOSYLMETHIONINE")
G.remove_node("OXYGEN-MOLECULE")
G.remove_node("ACETYL-COA")
G.remove_node("AMMONIUM")
G.remove_node("ADENOSYL-HOMO-CYS")
G.remove_node("Nucleoside-Triphosphates")
G.remove_node("Peptides-holder")
G.remove_node("RNA-Holder")
G.remove_node("Cytochromes-C-Oxidized")
G.remove_node("Cytochromes-C-Reduced")
G.remove_node("GDP")
G.remove_node("Ubiquitin-C-Terminal-Glycine")
G.remove_node("General-Protein-Substrates")

i = 0

while i < len(orfs):

    j=0
    k = 0
    while k < len(interactors[i]):

        if interactors_type[i][k] == "Positive Genetic":
            G.add_edge(orfs[i], interactors[i][k], label = interactors_type[i][k], weight = int(10))
          
        k+=1
    i+=1
    print (i /len(orfs))



nx.write_graphml(G,"citrate_posinteractions.xml")

0.0
0.0001399384270920795
0.000279876854184159
0.00041981528127623844
0.000559753708368318
0.0006996921354603975
0.0008396305625524769
0.0009795689896445563
0.001119507416736636
0.0012594458438287153
0.001399384270920795
0.0015393226980128744
0.0016792611251049538
0.0018191995521970334
0.0019591379792891126
0.0020990764063811922
0.002239014833473272
0.002378953260565351
0.0025188916876574307
0.0026588301147495103
0.00279876854184159
0.002938706968933669
0.0030786453960257487
0.0032185838231178284
0.0033585222502099076
0.003498460677301987
0.003638399104394067
0.003778337531486146
0.003918275958578225
0.004058214385670305
0.0041981528127623844
0.004338091239854464
0.004478029666946544
0.004617968094038623
0.004757906521130702
0.004897844948222782
0.005037783375314861
0.005177721802406941
0.005317660229499021
0.0054575986565911
0.00559753708368318
0.005737475510775259
0.005877413937867338
0.006017352364959418
0.0061572907920514975
0.006297229219143577
0.006437167646235657
0.0065771060733

0.2955499580184719
0.2956898964455639
0.295829834872656
0.2959697732997481
0.2961097117268402
0.2962496501539323
0.2963895885810244
0.2965295270081164
0.2966694654352085
0.2968094038623006
0.2969493422893927
0.29708928071648477
0.2972292191435768
0.2973691575706689
0.297509095997761
0.2976490344248531
0.29778897285194517
0.2979289112790372
0.2980688497061293
0.2982087881332214
0.29834872656031347
0.29848866498740556
0.2986286034144976
0.2987685418415897
0.2989084802686818
0.29904841869577387
0.29918835712286596
0.29932829554995805
0.2994682339770501
0.29960817240414217
0.29974811083123426
0.29988804925832635
0.30002798768541844
0.3001679261125105
0.30030786453960256
0.30044780296669465
0.30058774139378674
0.30072767982087883
0.30086761824797087
0.30100755667506296
0.30114749510215505
0.30128743352924714
0.30142737195633923
0.30156731038343126
0.30170724881052335
0.30184718723761544
0.30198712566470753
0.3021270640917996
0.3022670025188917
0.30240694094598375
0.30254687937307584
0.30268

0.5754268122026308
0.575566750629723
0.575706689056815
0.5758466274839071
0.5759865659109992
0.5761265043380912
0.5762664427651834
0.5764063811922754
0.5765463196193675
0.5766862580464596
0.5768261964735516
0.5769661349006437
0.5771060733277358
0.5772460117548279
0.57738595018192
0.577525888609012
0.5776658270361041
0.5778057654631962
0.5779457038902883
0.5780856423173804
0.5782255807444724
0.5783655191715645
0.5785054575986566
0.5786453960257487
0.5787853344528407
0.5789252728799328
0.5790652113070249
0.579205149734117
0.5793450881612091
0.5794850265883011
0.5796249650153932
0.5797649034424853
0.5799048418695774
0.5800447802966695
0.5801847187237615
0.5803246571508536
0.5804645955779457
0.5806045340050378
0.5807444724321299
0.5808844108592219
0.581024349286314
0.5811642877134061
0.5813042261404981
0.5814441645675903
0.5815841029946823
0.5817240414217745
0.5818639798488665
0.5820039182759585
0.5821438567030507
0.5822837951301427
0.5824237335572349
0.5825636719843269
0.5827036104114189


0.8650993562832354
0.8652392947103275
0.8653792331374195
0.8655191715645116
0.8656591099916037
0.8657990484186958
0.8659389868457879
0.8660789252728799
0.866218863699972
0.8663588021270641
0.8664987405541562
0.8666386789812482
0.8667786174083403
0.8669185558354324
0.8670584942625245
0.8671984326896166
0.8673383711167086
0.8674783095438007
0.8676182479708928
0.8677581863979849
0.867898124825077
0.868038063252169
0.8681780016792612
0.8683179401063532
0.8684578785334452
0.8685978169605374
0.8687377553876294
0.8688776938147216
0.8690176322418136
0.8691575706689056
0.8692975090959978
0.8694374475230898
0.869577385950182
0.869717324377274
0.869857262804366
0.8699972012314582
0.8701371396585502
0.8702770780856424
0.8704170165127344
0.8705569549398264
0.8706968933669186
0.8708368317940106
0.8709767702211028
0.8711167086481948
0.8712566470752868
0.871396585502379
0.871536523929471
0.8716764623565632
0.8718164007836552
0.8719563392107472
0.8720962776378394
0.8722362160649314
0.8723761544920235
0

0.05709487825356843
0.05723481668066051
0.05737475510775259
0.05751469353484467
0.05765463196193675
0.057794570389028826
0.05793450881612091
0.058074447243212986
0.05821438567030507
0.058354324097397145
0.05849426252448923
0.058634200951581304
0.05877413937867338
0.058914077805765463
0.05905401623285754
0.05919395465994962
0.0593338930870417
0.05947383151413378
0.05961376994122586
0.05975370836831794
0.05989364679541002
0.0600335852225021
0.06017352364959418
0.06031346207668626
0.060453400503778336
0.06059333893087042
0.060733277357962495
0.06087321578505458
0.061013154212146654
0.06115309263923874
0.061293031066330814
0.0614329694934229
0.06157290792051497
0.061712846347607056
0.06185278477469913
0.061992723201791215
0.06213266162888329
0.06227260005597537
0.06241253848306745
0.06255247691015953
0.0626924153372516
0.06283235376434369
0.06297229219143577
0.06311223061852785
0.06325216904561992
0.06339210747271201
0.06353204589980409
0.06367198432689616
0.06381192275398824
0.06395186118

...

0.3895885810243493
0.3897285194514414
0.38986845787853347
0.3900083963056255
0.3901483347327176
0.3902882731598097
0.3904282115869018
0.39056815001399386
0.3907080884410859
0.390848026868178
0.3909879652952701
0.39112790372236217
0.39126784214945426
0.3914077805765463
0.3915477190036384
0.3916876574307305
0.39182759585782256
0.39196753428491465
0.39210747271200674
0.3922474111390988
0.39238734956619087
0.39252728799328296
0.39266722642037505
0.39280716484746714
0.3929471032745592
0.39308704170165126
0.39322698012874335
0.39336691855583544
0.39350685698292753
0.39364679541001957
0.39378673383711166
0.39392667226420375
0.39406661069129584
0.39420654911838793
0.39434648754547996
0.39448642597257205
0.39462636439966414
0.39476630282675623
0.3949062412538483
0.3950461796809404
0.39518611810803245
0.39532605653512454
0.3954659949622166
0.3956059333893087
0.3957458718164008
0.39588581024349284
0.39602574867058493
0.396165687097677
0.3963056255247691
0.3964455639518612
0.39658550237895324
0.39

0.5076966134900643
0.5078365519171565
0.5079764903442485
0.5081164287713406
0.5082563671984327
0.5083963056255247
0.5085362440526169
0.5086761824797089
0.5088161209068011
0.5089560593338931
0.5090959977609851
0.5092359361880773
0.5093758746151693
0.5095158130422615
0.5096557514693535
0.5097956898964455
0.5099356283235377
0.5100755667506297
0.5102155051777219
0.5103554436048139
0.5104953820319059
0.5106353204589981
0.5107752588860901
0.5109151973131822
0.5110551357402743
0.5111950741673663
0.5113350125944585
0.5114749510215505
0.5116148894486426
0.5117548278757347
0.5118947663028267
0.5120347047299189
0.5121746431570109
0.512314581584103
0.5124545200111951
0.5125944584382871
0.5127343968653792
0.5128743352924713
0.5130142737195634
0.5131542121466555
0.5132941505737475
0.5134340890008396
0.5135740274279317
0.5137139658550238
0.5138539042821159
0.5139938427092079
0.5141337811363
0.5142737195633921
0.5144136579904842
0.5145535964175763
0.5146935348446684
0.5148334732717604
0.51497341169885

0.6041141897565071
0.6042541281835992
0.6043940666106913
0.6045340050377834
0.6046739434648755
0.6048138818919675
0.6049538203190596
0.6050937587461517
0.6052336971732438
0.6053736356003359
0.6055135740274279
0.60565351245452
0.6057934508816121
0.6059333893087042
0.6060733277357963
0.6062132661628883
0.6063532045899804
0.6064931430170725
0.6066330814441646
0.6067730198712566
0.6069129582983487
0.6070528967254408
0.6071928351525329
0.607332773579625
0.607472712006717
0.6076126504338091
0.6077525888609012
0.6078925272879933
0.6080324657150854
0.6081724041421774
0.6083123425692695
0.6084522809963616
0.6085922194234537
0.6087321578505458
0.6088720962776378
0.6090120347047299
0.609151973131822
0.609291911558914
0.6094318499860062
0.6095717884130982
0.6097117268401904
0.6098516652672824
0.6099916036943744
0.6101315421214666
0.6102714805485586
0.6104114189756508
0.6105513574027428
0.6106912958298348
0.610831234256927
0.610971172684019
0.6111111111111112
0.6112510495382032
0.6113909879652952
0

0.7483907080884411
0.7485306465155331
0.7486705849426253
0.7488105233697173
0.7489504617968094
0.7490904002239015
0.7492303386509935
0.7493702770780857
0.7495102155051777
0.7496501539322697
0.7497900923593619
0.7499300307864539
0.7500699692135461
0.7502099076406381
0.7503498460677303
0.7504897844948223
0.7506297229219143
0.7507696613490065
0.7509095997760985
0.7510495382031906
0.7511894766302827
0.7513294150573747
0.7514693534844669
0.7516092919115589
0.751749230338651
0.7518891687657431
0.7520291071928351
0.7521690456199273
0.7523089840470193
0.7524489224741114
0.7525888609012035
0.7527287993282955
0.7528687377553877
0.7530086761824797
0.7531486146095718
0.7532885530366639
0.7534284914637559
0.753568429890848
0.7537083683179401
0.7538483067450322
0.7539882451721243
0.7541281835992163
0.7542681220263084
0.7544080604534005
0.7545479988804926
0.7546879373075847
0.7548278757346767
0.7549678141617688
0.7551077525888609
0.755247691015953
0.755387629443045
0.7555275678701371
0.75566750629722

0.8674783095438007
0.8676182479708928
0.8677581863979849
0.867898124825077
0.868038063252169
0.8681780016792612
0.8683179401063532
0.8684578785334452
0.8685978169605374
0.8687377553876294
0.8688776938147216
0.8690176322418136
0.8691575706689056
0.8692975090959978
0.8694374475230898
0.869577385950182
0.869717324377274
0.869857262804366
0.8699972012314582
0.8701371396585502
0.8702770780856424
0.8704170165127344
0.8705569549398264
0.8706968933669186
0.8708368317940106
0.8709767702211028
0.8711167086481948
0.8712566470752868
0.871396585502379
0.871536523929471
0.8716764623565632
0.8718164007836552
0.8719563392107472
0.8720962776378394
0.8722362160649314
0.8723761544920235
0.8725160929191156
0.8726560313462076
0.8727959697732998
0.8729359082003918
0.8730758466274839
0.873215785054576
0.873355723481668
0.8734956619087602
0.8736356003358522
0.8737755387629443
0.8739154771900364
0.8740554156171285
0.8741953540442206
0.8743352924713126
0.8744752308984047
0.8746151693254968
0.8747551077525889
0.

In [42]:
nx.dijkstra_path_length(G1, 'TRS120', 'CIT')

33

In [52]:
G = nx.Graph()
G1 = nx.Graph()
i = 0
G.add_node('CIT', form = 'compound', distance = int(0))
while i < len(orfs):
    try:
        G.add_node(orfs[i], form = 'orf', name = names[i], distance = int(10*round(nx.dijkstra_path_length(G1, orfs[i], 'CIT'), 1)))
    except:
        G.add_node(orfs[i], form = 'orf', name = names[i], distance = -1)
        
    k = 0
    while k < len(left[i]):
        l = 0
        while l < len(left[i][k]):
            try:
                G.add_node(left[i][k][l], form = 'compound', distance = int(10*round(nx.dijkstra_path_length(G1, left[i][k][l], 'CIT'),1)))
            except:
                G.add_node(left[i][k][l], form = 'compound', distance = -1)
                
            G.add_edge(orfs[i], left[i][k][l], label = "backward_rxn", weight = int(10*0.5))
            #print("add edge")
            l+=1
        k+=1
    
          
    k = 0
    while k < len(right[i]):
        l = 0
        while l < len(right[i][k]):
            try:
                G.add_node(right[i][k][l], form = 'compound', distance = int(10*round(nx.dijkstra_path_length(G1, right[i][k][l], 'CIT'),1)))
            except:
                G.add_node(right[i][k][l], form = 'compound', distance = -1)
            G.add_edge(orfs[i], right[i][k][l], label = "forward_rxn", weight = int(10*0.5))
            #print("ADD EDGE")
            l+=1
        k+=1
    
    j=0
    #print(i/len(orfs))
    #nx.write_graphml(G,"metabolism_test.xml")
    print (i /len(orfs))
    i+=1
G.remove_node("PROTON")
G.remove_node("WATER")
G.remove_node("ATP")
G.remove_node("ADP")
G.remove_node("PPI")
G.remove_node("Pi")
G.remove_node("Protein-L-serine-or-L-threonine")
G.remove_node("Protein-Ser-or-Thr-phosphate")
G.remove_node("AMP")
G.remove_node("NAD")
G.remove_node("NADH")
G.remove_node("CO-A")
G.remove_node("NADP")
G.remove_node("NADPH")
G.remove_node("CARBON-DIOXIDE")
G.remove_node("GLT")
G.remove_node("S-ADENOSYLMETHIONINE")
G.remove_node("OXYGEN-MOLECULE")
G.remove_node("ACETYL-COA")
G.remove_node("AMMONIUM")
G.remove_node("ADENOSYL-HOMO-CYS")
G.remove_node("Nucleoside-Triphosphates")
G.remove_node("Peptides-holder")
G.remove_node("RNA-Holder")
G.remove_node("Cytochromes-C-Oxidized")
G.remove_node("Cytochromes-C-Reduced")
G.remove_node("GDP")
G.remove_node("Ubiquitin-C-Terminal-Glycine")
G.remove_node("General-Protein-Substrates")

i = 0

while i < len(orfs):
    k = 0
    while k < len(interactors[i]):
        if interactors_type[i][k] == "Positive Genetic":
            G.add_edge(names[i], interactors[i][k], label = interactors_type[i][k], weight = int(10))
 
        k+=1

    i+=1
    print (i /len(orfs))

import copy
G1 = copy.deepcopy(G)


G = nx.Graph()
i = 0
G.add_node('CIT', form = 'compound', distance = int(0))
while i < len(orfs):
    try:
        G.add_node(orfs[i], form = 'orf', name = names[i], distance = int(10*round(nx.dijkstra_path_length(G1, orfs[i], 'CIT'), 1)))
    except:
        G.add_node(orfs[i], form = 'orf', name = names[i], distance = -1)
    k = 0
    while k < len(left[i]):
        l = 0
        while l < len(left[i][k]):
            try:
                G.add_node(left[i][k][l], form = 'compound', distance = int(10*round(nx.dijkstra_path_length(G1, left[i][k][l], 'CIT'),1)))
            except:
                G.add_node(left[i][k][l], form = 'compound', distance = -1)
            G.add_edge(orfs[i], left[i][k][l], label = "backward_rxn", weight = 10*0.5)
            #print("add edge")
            l+=1
        k+=1
    
          
    k = 0
    while k < len(right[i]):
        l = 0
        while l < len(right[i][k]):
            try:
                G.add_node(right[i][k][l], form = 'compound', distance = int(10*round(nx.dijkstra_path_length(G1, right[i][k][l], 'CIT'),1)))
            except:
                G.add_node(right[i][k][l], form = 'compound', distance = -1)
            G.add_edge(orfs[i], right[i][k][l], label = "forward_rxn", weight = int(10*0.5))
            #print("ADD EDGE")
            l+=1
        k+=1
    
    j=0
    #print(i/len(orfs))
    #nx.write_graphml(G,"metabolism_test.xml")
    print (i /len(orfs))
    i+=1
G.remove_node("PROTON")
G.remove_node("WATER")
G.remove_node("ATP")
G.remove_node("ADP")
G.remove_node("PPI")
G.remove_node("Pi")
G.remove_node("Protein-L-serine-or-L-threonine")
G.remove_node("Protein-Ser-or-Thr-phosphate")
G.remove_node("AMP")
G.remove_node("NAD")
G.remove_node("NADH")
G.remove_node("CO-A")
G.remove_node("NADP")
G.remove_node("NADPH")
G.remove_node("CARBON-DIOXIDE")
G.remove_node("GLT")
G.remove_node("S-ADENOSYLMETHIONINE")
G.remove_node("OXYGEN-MOLECULE")
G.remove_node("ACETYL-COA")
G.remove_node("AMMONIUM")
G.remove_node("ADENOSYL-HOMO-CYS")
G.remove_node("Nucleoside-Triphosphates")
G.remove_node("Peptides-holder")
G.remove_node("RNA-Holder")
G.remove_node("Cytochromes-C-Oxidized")
G.remove_node("Cytochromes-C-Reduced")
G.remove_node("GDP")
G.remove_node("Ubiquitin-C-Terminal-Glycine")
G.remove_node("General-Protein-Substrates")

i = 0

while i < len(orfs):

    j=0
    k = 0
    while k < len(interactors[i]):

        if interactors_type[i][k] == "Positive Genetic":
            G.add_edge(orfs[i], interactors[i][k], label = interactors_type[i][k], weight = int(10))
          
        k+=1
    i+=1
    print (i /len(orfs))



nx.write_graphml(G,"citrate_posinteractions_equalweight.xml")

0.0
0.0001399384270920795
0.000279876854184159
0.00041981528127623844
0.000559753708368318
0.0006996921354603975
0.0008396305625524769
0.0009795689896445563
0.001119507416736636
0.0012594458438287153
0.001399384270920795
0.0015393226980128744
0.0016792611251049538
0.0018191995521970334
0.0019591379792891126
0.0020990764063811922
0.002239014833473272
0.002378953260565351
0.0025188916876574307
0.0026588301147495103
0.00279876854184159
0.002938706968933669
0.0030786453960257487
0.0032185838231178284
0.0033585222502099076
0.003498460677301987
0.003638399104394067
0.003778337531486146
0.003918275958578225
0.004058214385670305
0.0041981528127623844
0.004338091239854464
0.004478029666946544
0.004617968094038623
0.004757906521130702
0.004897844948222782
0.005037783375314861
0.005177721802406941
0.005317660229499021
0.0054575986565911
0.00559753708368318
0.005737475510775259
0.005877413937867338
0.006017352364959418
0.0061572907920514975
0.006297229219143577
0.006437167646235657
0.0065771060733

0.25608732157850544
0.25622726000559753
0.2563671984326896
0.2565071368597817
0.25664707528687375
0.25678701371396584
0.25692695214105793
0.25706689056815
0.2572068289952421
0.2573467674223342
0.25748670584942623
0.2576266442765183
0.2577665827036104
0.2579065211307025
0.2580464595577946
0.25818639798488663
0.2583263364119787
0.2584662748390708
0.2586062132661629
0.258746151693255
0.258886090120347
0.2590260285474391
0.2591659669745312
0.2593059054016233
0.2594458438287154
0.2595857822558074
0.2597257206828995
0.2598656591099916
0.2600055975370837
0.2601455359641758
0.26028547439126787
0.2604254128183599
0.260565351245452
0.2607052896725441
0.2608452280996362
0.26098516652672826
0.2611251049538203
0.2612650433809124
0.2614049818080045
0.26154492023509657
0.26168485866218866
0.2618247970892807
0.2619647355163728
0.26210467394346487
0.26224461237055696
0.26238455079764905
0.2625244892247411
0.2626644276518332
0.26280436607892527
0.26294430450601736
0.26308424293310945
0.26322418136020154

0.5202910719283516
0.5204310103554436
0.5205709487825357
0.5207108872096278
0.5208508256367198
0.520990764063812
0.521130702490904
0.5212706409179961
0.5214105793450882
0.5215505177721802
0.5216904561992723
0.5218303946263644
0.5219703330534565
0.5221102714805486
0.5222502099076406
0.5223901483347327
0.5225300867618248
0.5226700251889169
0.522809963616009
0.522949902043101
0.5230898404701931
0.5232297788972852
0.5233697173243773
0.5235096557514693
0.5236495941785614
0.5237895326056535
0.5239294710327456
0.5240694094598377
0.5242093478869297
0.5243492863140218
0.5244892247411139
0.524629163168206
0.5247691015952981
0.5249090400223901
0.5250489784494822
0.5251889168765743
0.5253288553036664
0.5254687937307585
0.5256087321578505
0.5257486705849427
0.5258886090120347
0.5260285474391267
0.5261684858662189
0.5263084242933109
0.5264483627204031
0.5265883011474951
0.5267282395745871
0.5268681780016793
0.5270081164287713
0.5271480548558635
0.5272879932829555
0.5274279317100475
0.527567870137139

0.8138818919675342
0.8140218303946264
0.8141617688217184
0.8143017072488106
0.8144416456759026
0.8145815841029946
0.8147215225300868
0.8148614609571788
0.815001399384271
0.815141337811363
0.815281276238455
0.8154212146655472
0.8155611530926392
0.8157010915197314
0.8158410299468234
0.8159809683739154
0.8161209068010076
0.8162608452280996
0.8164007836551918
0.8165407220822838
0.8166806605093758
0.816820598936468
0.81696053736356
0.8171004757906521
0.8172404142177442
0.8173803526448362
0.8175202910719284
0.8176602294990204
0.8178001679261125
0.8179401063532046
0.8180800447802967
0.8182199832073888
0.8183599216344808
0.8184998600615729
0.818639798488665
0.8187797369157571
0.8189196753428492
0.8190596137699412
0.8191995521970333
0.8193394906241254
0.8194794290512175
0.8196193674783095
0.8197593059054016
0.8198992443324937
0.8200391827595858
0.8201791211866779
0.8203190596137699
0.820458998040862
0.8205989364679541
0.8207388748950462
0.8208788133221383
0.8210187517492303
0.8211586901763224
0

0.04869857262804366
0.04883851105513574
0.04897844948222782
0.0491183879093199
0.04925832633641198
0.049398264763504056
0.04953820319059614
0.049678141617688215
0.0498180800447803
0.049958018471872374
0.05009795689896446
0.050237895326056534
0.05037783375314862
0.05051777218024069
0.050657710607332776
0.05079764903442485
0.050937587461516935
0.05107752588860901
0.051217464315701094
0.05135740274279317
0.051497341169885254
0.05163727959697733
0.051777218024069406
0.05191715645116149
0.052057094878253565
0.05219703330534565
0.052336971732437725
0.05247691015952981
0.052616848586621884
0.05275678701371397
0.05289672544080604
0.053036663867898126
0.0531766022949902
0.053316540722082285
0.05345647914917436
0.053596417576266445
0.05373635600335852
0.053876294430450604
0.05401623285754268
0.05415617128463476
0.05429610971172684
0.05443604813881892
0.054575986565911
0.05471592499300308
0.05485586342009516
0.05499580184718724
0.05513574027427932
0.05527567870137139
0.055415617128463476
0.055555

...

0.38706968933669184
0.3872096277637839
0.387349566190876
0.3874895046179681
0.3876294430450602
0.38776938147215223
0.3879093198992443
0.3880492583263364
0.3881891967534285
0.3883291351805206
0.3884690736076126
0.3886090120347047
0.3887489504617968
0.3888888888888889
0.389028827315981
0.3891687657430731
0.3893087041701651
0.3894486425972572
0.3895885810243493
0.3897285194514414
0.38986845787853347
0.3900083963056255
0.3901483347327176
0.3902882731598097
0.3904282115869018
0.39056815001399386
0.3907080884410859
0.390848026868178
0.3909879652952701
0.39112790372236217
0.39126784214945426
0.3914077805765463
0.3915477190036384
0.3916876574307305
0.39182759585782256
0.39196753428491465
0.39210747271200674
0.3922474111390988
0.39238734956619087
0.39252728799328296
0.39266722642037505
0.39280716484746714
0.3929471032745592
0.39308704170165126
0.39322698012874335
0.39336691855583544
0.39350685698292753
0.39364679541001957
0.39378673383711166
0.39392667226420375
0.39406661069129584
0.39420654911

0.5321858382311783
0.5323257766582704
0.5324657150853624
0.5326056535124545
0.5327455919395466
0.5328855303666387
0.5330254687937308
0.5331654072208228
0.5333053456479149
0.533445284075007
0.533585222502099
0.5337251609291912
0.5338650993562832
0.5340050377833753
0.5341449762104674
0.5342849146375594
0.5344248530646516
0.5345647914917436
0.5347047299188357
0.5348446683459278
0.5349846067730198
0.535124545200112
0.535264483627204
0.5354044220542961
0.5355443604813882
0.5356842989084802
0.5358242373355724
0.5359641757626644
0.5361041141897565
0.5362440526168486
0.5363839910439406
0.5365239294710328
0.5366638678981248
0.536803806325217
0.536943744752309
0.537083683179401
0.5372236216064932
0.5373635600335852
0.5375034984606774
0.5376434368877694
0.5377833753148614
0.5379233137419536
0.5380632521690456
0.5382031905961377
0.5383431290232298
0.5384830674503218
0.538623005877414
0.538762944304506
0.5389028827315981
0.5390428211586902
0.5391827595857822
0.5393226980128744
0.5394626364399664
0.

0.6533725160929191
0.6535124545200112
0.6536523929471033
0.6537923313741953
0.6539322698012874
0.6540722082283795
0.6542121466554716
0.6543520850825637
0.6544920235096557
0.6546319619367479
0.6547719003638399
0.654911838790932
0.6550517772180241
0.6551917156451161
0.6553316540722083
0.6554715924993003
0.6556115309263923
0.6557514693534845
0.6558914077805765
0.6560313462076687
0.6561712846347607
0.6563112230618527
0.6564511614889449
0.6565910999160369
0.6567310383431291
0.6568709767702211
0.6570109151973131
0.6571508536244053
0.6572907920514973
0.6574307304785895
0.6575706689056815
0.6577106073327735
0.6578505457598657
0.6579904841869577
0.6581304226140499
0.6582703610411419
0.6584102994682339
0.6585502378953261
0.6586901763224181
0.6588301147495103
0.6589700531766023
0.6591099916036943
0.6592499300307865
0.6593898684578785
0.6595298068849706
0.6596697453120627
0.6598096837391547
0.6599496221662469
0.6600895605933389
0.660229499020431
0.6603694374475231
0.6605093758746152
0.660649314301

0.7903722362160649
0.790512174643157
0.7906521130702491
0.7907920514973412
0.7909319899244333
0.7910719283515253
0.7912118667786174
0.7913518052057095
0.7914917436328016
0.7916316820598936
0.7917716204869857
0.7919115589140778
0.7920514973411699
0.792191435768262
0.792331374195354
0.7924713126224461
0.7926112510495382
0.7927511894766303
0.7928911279037224
0.7930310663308144
0.7931710047579065
0.7933109431849986
0.7934508816120907
0.7935908200391828
0.7937307584662748
0.7938706968933669
0.794010635320459
0.794150573747551
0.7942905121746432
0.7944304506017352
0.7945703890288273
0.7947103274559194
0.7948502658830114
0.7949902043101036
0.7951301427371956
0.7952700811642878
0.7954100195913798
0.7955499580184718
0.795689896445564
0.795829834872656
0.7959697732997482
0.7961097117268402
0.7962496501539322
0.7963895885810244
0.7965295270081164
0.7966694654352086
0.7968094038623006
0.7969493422893926
0.7970892807164848
0.7972292191435768
0.797369157570669
0.797509095997761
0.797649034424853
0.7

0.9033025468793731
0.9034424853064652
0.9035824237335572
0.9037223621606493
0.9038623005877414
0.9040022390148335
0.9041421774419256
0.9042821158690176
0.9044220542961097
0.9045619927232018
0.9047019311502938
0.904841869577386
0.904981808004478
0.9051217464315701
0.9052616848586622
0.9054016232857542
0.9055415617128464
0.9056815001399384
0.9058214385670305
0.9059613769941226
0.9061013154212146
0.9062412538483068
0.9063811922753988
0.9065211307024909
0.906661069129583
0.906801007556675
0.9069409459837672
0.9070808844108592
0.9072208228379512
0.9073607612650434
0.9075006996921354
0.9076406381192276
0.9077805765463196
0.9079205149734118
0.9080604534005038
0.9082003918275958
0.908340330254688
0.90848026868178
0.9086202071088721
0.9087601455359642
0.9089000839630562
0.9090400223901484
0.9091799608172404
0.9093198992443325
0.9094598376714246
0.9095997760985166
0.9097397145256088
0.9098796529527008
0.9100195913797929
0.910159529806885
0.910299468233977
0.9104394066610692
0.9105793450881612
0.

In [53]:
for value in G.nodes:
    try:
        G.nodes[value]['distance'] = int(10*round(nx.dijkstra_path_length(G, value, 'CIT'),1))
    except:
        G.nodes[value]['distance'] = -1

In [54]:
nx.write_graphml(G,"citrate_posinteractions_equalweight.xml")

In [ ]:
orfs = pd.read_csv('final_citrate_pos_interactions_orfs.csv')
sns.countplot(data = orfs, x = "distance")
plt.savefig('distance_chart.svg'